In [1]:

import openpyxl
import glob
import os
from openpyxl import Workbook
import numpy as np
import xlwings as xw
from openpyxl.styles import PatternFill

In [2]:
# Please enter the path where the Excel file you want to narrow down is saved.*and Please change user name!!!!
folderpath = "/Users/cerasus/Desktop/VOC/0.textから変換済み"
# Please enter the destination path of the Excel file you wish to save.
save_path = "/Users/cerasus/Desktop/VOC/1.絞り込み済み"



#Row number of the cell where the peak begins
peak_cell_start = 24


same_substance_col = 'E'
possible_col = 'C'


#How far is the peak? 
peak_range = 10



In [3]:
"""
1.Remove compounds with the same name and narrow down the list based on probability.
"""

files = sorted(glob.glob(folderpath + "/*.xlsx"))

num=1

for file in files:
    filename, ext = os.path.splitext(os.path.basename(file))
    wb = openpyxl.load_workbook(file)
    ws = wb.worksheets[0]
    #peakスタートの位置を揃える（０）
    if ws.cell(peak_range+13, 1).value != 'Spectrum#':
        for cell in ws.iter_rows(min_col=1, max_col=1):
            if cell[0].value == 'Spectrum#':
                spectal_cell = cell[0]
                between= peak_range + 13 - spectal_cell.row 
                ws.insert_rows(spectal_cell.row-2, between)
                break
    #peakごとにリスト分けする（１）
    
    peak_num=1
    peak_cell_s = peak_cell_start
    for i in range(1,peak_range+1):
        peak_list = []
        sublist =[]
        flg = False
        for peak in ws['A'+str(peak_cell_s) +':A'+ str(ws.max_row)]: 
            if peak[0].value == peak_num:
                    sublist.append(peak[0])
            if flg:
                break
            elif peak[0].value == peak_num+1:
                flg = True
        #欠損ファイルは途中で止める
        if sublist == []:
            break
    
        # 同じ物質の削除の用意をする（２）
        delete_list = []
        peak_s=sublist[0].row
        peak_e=sublist[-1].row
        peak_re=sublist[-1].row
        sec = peak_s + 1
        for onazi in range(peak_s, peak_e):
            for onazi2 in range(sec,peak_e + 1):
                if ws[same_substance_col + str(onazi)].value == ws[same_substance_col + str(onazi2)].value:   #あとで列設定
                    delete_list.append(ws[same_substance_col + str(onazi2)].row)
            sec += 1
    
        #削除（３）
        dlt_list=sorted(set((delete_list)))
        dlt_num = 0
        dlt_sum=0

        for dlt in dlt_list:
            ws.delete_rows(dlt - dlt_num)
            dlt_num += 1
            dlt_sum+=1

        delete_list.clear()
        sublist.clear()
        #削除したため、ピークリストの再取得
        peak_list = []
        sublist =[]
        flg = False
        for peak in ws['A'+str(peak_cell_s) +':A'+ str(ws.max_row)]: #peakのセル、あとで設定
            if peak[0].value == peak_num:
                sublist.append(peak[0])
            if flg:
                break
            elif peak[0].value == peak_num+1:
                flg = True
        peak_num+=1


        peak_s=sublist[0].row
        peak_e=sublist[-1].row
        
        # 確率の重複確認（４）
        head_list =[] #重複を確認するリストを用意
        head_list.clear()
        for fin_del_tuple in ws[possible_col+str(peak_s)+':'+ possible_col +str(peak_s+7)]:
            head_list.append(fin_del_tuple[0].value)
        
        
        if head_list[2] == head_list[6]:
            final_del = ws[possible_col+str(peak_s+7)].row
        elif head_list[2] == head_list[5]:
            final_del = ws[possible_col+str(peak_s+6)].row
        elif head_list[2] == head_list[4]:
            final_del = ws[possible_col+str(peak_s+5)].row
        elif head_list[2] == head_list[3]:
            final_del = ws[possible_col+str(peak_s+4)].row
        else:
            final_del = ws[possible_col+str(peak_s+3)].row

        #最終削除（５）
        dlt_num = 0
        for dlt in range(final_del, peak_e+1):
            ws.delete_rows(dlt - dlt_num)
            dlt_num += 1
            dlt_sum+=1
        peak_re -= dlt_sum
        peak_cell_s = peak_re
        peak_list.clear()
        sublist.clear()
        dlt_list.clear()
    ws.insert_cols(3, 2)
    ws.delete_rows(1)
    ws.title=filename
    wb.save(save_path + '/'+ filename + ".xlsx" )
        
        
        
        
    

wb.close()    
    
print('Filtering is complete.')
print('Enter the IFERROR function.')

絞り込みが終了しました
IFERROR関数を入力してください


In [4]:
"""
２.The file will be overwritten and saved.
  If you want to save a new file, erase save_path and enter the path name.
  Attention! Close all open Excel files. If you have an Excel file open, close it.
"""
IFError_load_path=save_path

files = sorted(glob.glob(IFError_load_path + "/*.xlsx"))


for fil in files:
    xb=xw.Book(fil)
    max_row = xw.sheets[0].range('A'+str(peak_cell_start)).end('down').last_cell.row
    for iferror in range(peak_cell_start-1, max_row+1):
        xb.sheets[0].range((iferror,3)).formula= "=IFERROR(VLOOKUP(A"+str(iferror)+",$A$9:$H$"+ str(8+peak_range) +",2,FALSE),"")"
    
    for iferror in range(peak_cell_start-1, max_row+1):
        xb.sheets[0].range((iferror,4)).formula = '=IFERROR(VLOOKUP(A'+str(iferror)+',$A$9:$H$'+ str(8+peak_range) +',8,FALSE),"")'
    
    xb.save()
    app=xw.apps.active
    app.quit()

print('Finished')


終了しました


In [13]:
"""
３.Combine them into a single file.
　Enter the path name and run the program.
 In this process　do not use Rt.time for alignment.
"""

# Please enter the path where the Excel file you want to summarize is saved.
folderpath = "/Users/cerasus/Desktop/VOC/1.filtered"
# Enter the destination path of the Excel file you want to save.
save_path = "/Users/cerasus/Desktop/VOC/2.summary"

files = sorted(glob.glob(folderpath + "/*.xlsx"))

wb2 = Workbook()
new_ws = wb2.active

initial_y, initial_x, second_x =1, 5, 5
y_header=1
file_num = 0

header_flg=True

for file in files:
    filename, ext = os.path.splitext(os.path.basename(file))
    wb3 = openpyxl.load_workbook(file, data_only=True)
    ws3 = wb3.worksheets[0]
    file_num +=1
    peak_num=1
    peak_cell_s = peak_cell_start-1
    for i in range(1,peak_range+1):
            peak_list = []
            sublist =[]
            flg = False
            for peak in ws3['A'+str(peak_cell_s) +':A'+ str(ws3.max_row)]: 
                if peak[0].value == peak_num:
                    sublist.append(peak[0])
                if flg:
                    break
                elif peak[0].value == peak_num+1:
                    flg = True
            peak_num+=1
            if sublist == []:
                break
            peak_s=sublist[0].row
            peak_e=sublist[-1].row
            peak_cell_s = peak_e
#各セルから貼り付け（六セルごとに
            
            y_s = initial_y
            x_s = second_x
            x_header=5
            new_ws.cell(x_s, y_s).value = ws3.cell(peak_s, 1).value #spectol
            new_ws.cell(x_s, y_s+1).value = ws3.cell(peak_s, 3).value#peak
            new_ws.cell(x_s, y_s+2).value = ws3.cell(peak_s, 4).value #area

            new_ws.cell(x_header-1, y_s).value = ws3.cell(peak_cell_start-1,1).value #spectol header
            new_ws.cell(x_header-1, y_s+1).value = 'Rt.time'
            new_ws.cell(x_header-1, y_s+2).value = '面積'
            new_ws.cell(x_header-1, y_header+3).value = ws3.cell(peak_cell_start-2, 6).value #CAS
            new_ws.cell(x_header-1, y_header+4).value = ws3.cell(peak_cell_start-2, 7).value#Name
            new_ws.cell(x_header-1, y_header+5).value = ws3.cell(peak_cell_start-2, 5).value#SI

            new_ws.cell(x_header-4, y_header+2).value = filename
            
            for CAS in ws3['F'+str(peak_s)+':'+'F'+str(peak_e)]:
                new_ws.cell(x_s,y_s +3).value = CAS[0].value
                x_s += 1

            x_s = initial_x
            for Name in ws3['G'+str(peak_s)+':'+'G'+str(peak_e)]:
                new_ws.cell(x_s,y_s +4).value = Name[0].value
                x_s += 1

            x_s = initial_x
            for SI in ws3['E'+str(peak_s)+':'+'E'+str(peak_e)]:
                new_ws.cell(x_s,y_s +5).value = SI[0].value
                x_s += 1



            second_x+= 9
            initial_x+=9
    second_x=5
    initial_x=5
    initial_y+=6 
    y_header+=6

wb2.save(save_path + '/'+ 'gathering_data' + ".xlsx" )

print('終了しました')

          






終了しました


In [14]:
#Enter the pathname of the file to be aligned with retention time
file="/Users/cerasus/Desktop/VOC/2.summary/gathering_data.xlsx"

#saveing Path name
savepath="/Users/cerasus/Desktop/VOC/2.summary/comparision.xlsx"
originalfilespath="/Users/cerasus/Desktop/VOC/0.Converted _file"


#Enter the CAS number to be colored
given_CAS = ['928 - 96 - 1',
             '3173 - 53 - 3']

In [15]:
"""
３.Align the summarized files.
"""

def getRt_CAS_cols(cols_list):
    for i_list in [Rt_time_list,normal_list,small_list,big_list,Rt_time_value,CAS_val,CAS_list]:
        i_list.clear()
    for Rt_col in cols_list:
        Rt_time_list.append(ws5.cell(Rt_x,Rt_col))
        if ws5.cell(Rt_x,Rt_col).value != None:
            Rt_time_value.append(ws5.cell(Rt_x,Rt_col).value)
        CAS_list.append(ws5.cell(Rt_x,Rt_col+2))
        if ws5.cell(Rt_x,Rt_col+3).value != None:
            CAS_val.append(ws5.cell(Rt_x,Rt_col+2).value)
            

wb5 = openpyxl.load_workbook(file, data_only=True)
ws5 = wb5.worksheets[0]
Rt_cols_list=[]
for Rt_time_rows in ws5.iter_rows(min_row=4, max_row=4):
    for Rt_time_cell in Rt_time_rows:
        if Rt_time_cell.value == 'Rt.time':
            Rt_cols_list.append(Rt_time_cell.column)

Rt_time_list,normal_list,small_list,big_list,Rt_time_value,CAS_val,CAS_list=[],[],[],[],[],[],[]
x, Rt_x=5, 5
Rt_flg, b_flg, s_flg=True, False ,False

getRt_CAS_cols(Rt_cols_list)


while len(Rt_time_value) != 0:
    #Rttimeをリストに分ける
    CAS_num=0
    for Rt_time in Rt_time_list:
        if Rt_time.value == None:
            pass
        elif max(Rt_time_value) - 0.5 <= Rt_time.value <= max(Rt_time_value):
            big_list.append(Rt_time)
            b_flg=True
            CAS_num+=1
        elif CAS_val[CAS_num]==CAS_val[Rt_time_value.index(max(Rt_time_value))]:#範囲外でも同じCASなら同じとみなす
                big_list.append(Rt_time)
                CAS_num+=1
        elif min(Rt_time_value)<= Rt_time.value <= min(Rt_time_value) + 0.5:
            small_list.append(Rt_time)
            s_flg=True
            CAS_num+=1
        elif CAS_val[CAS_num]==CAS_val[Rt_time_value.index(min(Rt_time_value))]:#範囲外でも同じCASなら同じとみなす
                    small_list.append(Rt_time)
                    CAS_num+=1
        else :
            normal_list.append(Rt_time)
            CAS_num+=1

    s_flg=False
    b_flg=False
    #Rt.timeが小さいリスト以外を動かす
    if len(small_list) != 0:
        for b_sum in big_list:
            normal_list.append(b_sum)
        for move_col in normal_list:
            start_cel=ws5.cell(x,move_col.column-1).coordinate
            end_cel= ws5.cell(ws5.max_row,move_col.column+4).coordinate
            ws5.move_range(start_cel+':'+end_cel, rows=9)
    elif len(big_list) != 0 and len(normal_list) != 0:
        for move_col in normal_list:
            start_cel=ws5.cell(x,move_col.column-1).coordinate
            end_cel= ws5.cell(ws5.max_row,move_col.column+4).coordinate
            ws5.move_range(start_cel+':'+end_cel, rows=9)
    else:
        pass
    x += 9
    Rt_x +=9
    Rt_flg= True
    getRt_CAS_cols(Rt_cols_list)
wb.save(savepath)    
print('Finished!')

終了しました


In [21]:
"""
Extra . Add the original data (sheet) on the aligned file.
"""
#比較.xlsxのパス
copy_path ='/Users/cerasus/Desktop/VOC/2.summary/comparision.xlsx'
xb2=xw.Book(copy_path)
sheet = xb2.sheets[-1]
#処理前のパス
files = sorted(glob.glob(originalfilespath + "/*.xlsx"))

#API機能を使用するため、WindowsとMacでコードを変える必要がある
for fil in files:
    xb=xw.Book(fil)
    xb.sheets[0].api.Copy(Before=sheet.api)
    xb.close()
    sheet = xb2.sheets[-1]
#xb2.save()
xb2.save('/Users/cerasus/Desktop/VOC/2.summary/comparision2.xlsx')
xb2.close()

print('終了しました')


終了しました


In [16]:
wb4 = openpyxl.load_workbook(copy_path, data_only=True)
ws4 = wb4.worksheets[0]



#CASと書かれたセルの座標を取得し、リストに作成（２）_整列させる
CAS_list = []
CAS_list.clear()
for CAS_rows in ws4.iter_rows(min_row=4, max_row=4):
    for CAS_row in CAS_rows:
        if CAS_row.value == 'CAS #':
            CAS_list.append(CAS_row.column)
            
#重複リストと（２）を用いて、色を塗る
fill = PatternFill(patternType='solid', fgColor='FFFF66')

# write in sheet
for fill_col in CAS_list:
    for fill_cells in ws4.iter_cols(min_col=fill_col, max_col=fill_col):
        for fill_cell in fill_cells:
            if fill_cell.value in given_CAS:
                ws4[fill_cell.coordinate].fill = fill
wb4.save(savepath)
wb4.close()

print('Finished!')

終了しました


In [17]:
"""
If the URL is not localhost:8888, an error may occur.
Please reboot your PC.
"""

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-17-d7345d920368>, line 3)